In [1]:
%pip install tiktoken
%pip install openai
%pip install textract
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install plotly
%pip install scipy
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 KB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 KB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 KB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 KB 77.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 라이브러리 임포트

In [2]:
import os
import requests
import numpy as np
import pandas as pd
from typing import Iterator
import tiktoken
import textract

# 논문 파일 리스트 구하기

In [3]:
import os

data_path = "data/sample"
pdf_files = [x for x in os.listdir(data_path)]

print(len(pdf_files))
print(pdf_files[:3])

20
['jkma-2023-66-1-11.pdf', 'jkma-2023-66-1-19.pdf', 'jkma-2023-66-1-31.pdf']


In [4]:
# pdf_files = pdf_files[:2] 

# 파일을 조각(chunk)로 쪼개고 임베딩해 두기

In [10]:
import openai
openai.api_key = ""

In [11]:
import tiktoken

TEXT_EMBEDDING_CHUNK_SIZE = 2000
EMBEDDINGS_MODEL = "text-embedding-ada-002"
tokenizer = tiktoken.get_encoding("cl100k_base")

embedding_storage = []

# 긴 문자열을 최대 길이 chunk_size의 토큰들로 자르고, 토큰을 반환한다.
def chunks(text, chunk_size=TEXT_EMBEDDING_CHUNK_SIZE):
    """Yield successive chunk_size chunks from text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # 토큰 길이의 0.5 ~ 1.5 사이에서 문장의 끝을 찾는다.
        j = min(i + int(1.5 * chunk_size), len(tokens))
        while j > i + int(0.5 * chunk_size):
            # 디코딩 해서 마침표 혹은 줄 바꿈인지 확인한다.
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # 문장 끝을 못찾으면 chunk_size로 한다.
        if j == i + int(0.5 * chunk_size):
            j = min(i + chunk_size, len(tokens))
        yield tokens[i:j]
        i = j

def get_embeddings(text_array):
    embedding_objects = openai.Embedding.create(input=text_array, model=EMBEDDINGS_MODEL)["data"]
    return [embedding_object["embedding"] for embedding_object in embedding_objects]

    
def create_embeddings_for_text(text):
    # 최대길이 TEXT_EMBEDDING_CHUNK_SIZE의 토큰 리스트로 변환. 쪼개진 개별을 chunk라 한다.
    token_chunks = list(chunks(text))
    # 각 토큰 리스트를 text 리스트로 변환
    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks]

    # 전체 chunk들의 임베딩을 구하고
    embedding_chunks = get_embeddings(text_chunks)
    return text_chunks, embedding_chunks


def handle_file_string(filename, file_content_string):
    
    # 파일 내용에서 라인 변경, 더블 스페이스, 세미 콜론을 삭제
    clean_file_content_string = " ".join(file_content_string.replace("\n", "; ").replace(";", " ").split())
        
    try:
        text_chunks, embedding_chunks = create_embeddings_for_text(clean_file_content_string)
        print(f"[INFO] Embedded into {len(text_chunks)} chunks")
    except Exception as e:
        print("[handle_file_string] Error creating embedding: {}".format(e))

    for i, (text_chunk, embedding_chunk) in enumerate(zip(text_chunks, embedding_chunks)):
        embedding_storage.append({"embedding":embedding_chunk, "filename": filename, "chunk_index":i, "text":text_chunk })


In [12]:

# Process each PDF file and prepare for embedding
for i, pdf_file in enumerate(pdf_files):
    
    pdf_path = os.path.join(data_path, pdf_file)
    print(f"[INFO] {i}/{len(pdf_files)}. processing paper : {pdf_path}\n")
    
    text = textract.process(pdf_path, method='pdfminer')
    print(text.decode("utf-8")[0:200])
    print()

    handle_file_string(pdf_file, text.decode("utf-8"))
    print()
    print("============================================================")


[INFO] 0/20. processing paper : data/sample/jkma-2023-66-1-11.pdf

FOCUSED ISSUE OF THIS MONTH 
J Korean Med Assoc 2023 January; 66(1):11-18

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.1.11

삼차신경통의 진단과 치료
삼차신경통의 진단과 치료

박 창 규·임 승 훈·박 봉 진


[INFO] Embedded into 5 chunks

[INFO] 1/20. processing paper : data/sample/jkma-2023-66-1-19.pdf

 특집2

FOCUSED ISSUE OF THIS MONTH 
J Korean Med Assoc 2023 January; 66(1):19-30

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.1.19

혀인두신경통의 진단과 치료
혀인두신경통의 진단과 치료

안 영 환1,2


[INFO] Embedded into 8 chunks

[INFO] 2/20. processing paper : data/sample/jkma-2023-66-1-31.pdf

 특집3

FOCUSED ISSUE OF THIS MONTH 
J Korean Med Assoc 2023 January; 66(1):31-40

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.1.31

후두신경통의 진단과 치료: 큰뒤통수신경 포착증후
후두신경통의 진단과 치료

[INFO] Embedded into 7 chunks

[INFO] 3/20. processing paper : data/sample/jkma-2023-66-1-41.pdf

CONTINUING EDUCATION COLUMN 
J Ko

In [13]:
from pprint import pprint

print(len(embedding_storage))
print(embedding_storage[0]["embedding"][:10])
print(len(embedding_storage[0]["embedding"]))
print(embedding_storage[0]["filename"])
print(embedding_storage[0]["chunk_index"])
print(embedding_storage[0]["text"])

110
[-0.030031222850084305, 0.012291058897972107, 0.06017115339636803, -0.010055703110992908, -0.015613517723977566, 0.0042668795213103294, -0.026158418506383896, -0.015599928796291351, -0.02742217481136322, -0.013813002966344357]
1536
jkma-2023-66-1-11.pdf
0
FOCUSED ISSUE OF THIS MONTH J Korean Med Assoc 2023 January 66(1):11-18 pISSN 1975-8456 / eISSN 2093-5951 https://doi.org/10.5124/jkma.2023.66.1.11 삼차신경통의 진단과 치료 삼차신경통의 진단과 치료 박 창 규·임 승 훈·박 봉 진 경희대학교 의과대학 신경외과학교실 Diagnosis and treatment of trigeminal neuralgia Chang Kyu Park, MD · Seung Hoon Lim, MD · Bong Jin Park, MD Department of Neurosurgery, College of Medicine, Kyung Hee University, Seoul, Korea Background: Trigeminal neuralgia is a sudden and painful facial condition that is triggered by activities of daily living. The pain is debilitating and patients are often unable to perform routine daily tasks such as washing their face, shaving, and brushing their teeth, which in turn has an impact on their social life and mental wel

# 가장 가까운 조각(chunk) 구하기

In [14]:
from openai.embeddings_utils import distances_from_embeddings, indices_of_nearest_neighbors_from_distances

def find_closest_n_index(embedding_storage, embedded_query, n=3):
    embeddings = [entity["embedding"] for entity in embedding_storage]
    distances = distances_from_embeddings(embedded_query, embeddings, distance_metric="cosine")
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)
    return indices_of_nearest_neighbors[:n]


def find_closest_n_chunk(embedding_storage, query, n=3):
    embedded_query = get_embeddings(query)[0]
    closest_indexes = find_closest_n_index(embedding_storage, embedded_query, n)
    return [embedding_storage[i] for i in closest_indexes]


In [15]:
question = "코로나19 증후군의 증상은?"
queryed = find_closest_n_chunk(embedding_storage, question, n=1)[0]

In [16]:
print(queryed["filename"])
print(queryed["chunk_index"])
print(queryed["text"])


jkma-2023-66-3-200.pdf
2
2%로 가장 낮았다. 평소 호흡곤란, 운동 호흡곤란, 가슴 답답함), 후·미각계, Figure 1에서는 확진일에 따른 전체 후유증 발생률과 각 육군 만성 코로나19 증후군 203 J Korean Med Assoc 2023 March 66(3):200-208 Figure 1. Trends of the incidence rate of post-COVID-19 conditions by type. 계열별 발생률을 시계열적으로 제시하였다. 확진자들의 전체 시간이 지날수록 발생률이 상승하여 5월 4주에 이르러서는 후유증 발생률은 델타 변이종 우세기에 50.7%로 가장 높았으 기침 31.6%, 가래 26.9%에 이르렀다. 며, 오미크론 변이종 우세기인 2022년 1월 4주에는 19.7%로 가장 낮아졌다가 이후 30%에서 40%로 서서히 증가하는 경향 이 나타났다. 계열별로는 신경정신계 후유증이 델타 우세기 고찰 에 27.4%, 2022년 1주차에 29.0%로 가장 발생률이 높았으나 이후 낮아져 15% 수준을 유지하였다. 반면 호흡기계 후유증 코로나19의 대유행으로 2022년 현재 우리나라 국민의 절 은 2022년 1월 4주부터 14.1%로 신경정신계 후유증보다 발 반 이상이 코로나19를 경험하였다는 점을 감안하면, ’코로나 생률이 높아진 이후 점차 발생률이 증가하여 5월 1주 37.3% 19 후유증’에 대한 진료지침 및 관리 정책의 개발은 공중보 에 이르렀다. 후각/미각장애의 경우 델타 시기에는 21.1%로 건의 관점에서 중요한 문제라고 할 수 있다. 문제는 이러한 높은 발생률을 보였으나 나머지 시기에는 그렇지 않았다. ‘코로나19 후유증’에 대한 명확한 개념과 정의가 부재하였다 Figure 2는 확진일에 따른 신경정신계 후유증 내의 개별 는 것이다. 코로나19 후유증(long COVID)이라는 명칭 외에 항목별 발생률을 시계열적으로 나타낸 것이다. 모든 시기 도 post COVID-19 condition [5], post-ac

# 구한 조각 내용으로 문의하기

In [32]:
act_as = f"""You are helpful QnA engine.
You will be provided with text delimited by triple quotes and question. 
Step 1. find the text part from the given text which is used to reason the answer of question
Step 2. answer the question in Korean and show the text part with quotation mark.
If not base found just say 'could not answer'. 
"""

text_and_question = f"""
Text:
```{queryed["text"]}```

Question:
```{question}```
"""


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": act_as},
        {"role": "user", "content": text_and_question},
    ],
    temperature=0,
)

print("질문 : ", question)
print("근거 문서 파일 이름 : ", queryed["filename"])
print("근거 문서 조각 인덱스 : ", queryed["chunk_index"])
print("근거 문서 내용 : ", queryed["text"])
print("답변 : ", response['choices'][0]['message']['content'])

질문 :  코로나19 증후군의 증상은?
근거 문서 파일 이름 :  jkma-2023-66-3-200.pdf
근거 문서 조각 인덱스 :  2
근거 문서 내용 :  2%로 가장 낮았다. 평소 호흡곤란, 운동 호흡곤란, 가슴 답답함), 후·미각계, Figure 1에서는 확진일에 따른 전체 후유증 발생률과 각 육군 만성 코로나19 증후군 203 J Korean Med Assoc 2023 March 66(3):200-208 Figure 1. Trends of the incidence rate of post-COVID-19 conditions by type. 계열별 발생률을 시계열적으로 제시하였다. 확진자들의 전체 시간이 지날수록 발생률이 상승하여 5월 4주에 이르러서는 후유증 발생률은 델타 변이종 우세기에 50.7%로 가장 높았으 기침 31.6%, 가래 26.9%에 이르렀다. 며, 오미크론 변이종 우세기인 2022년 1월 4주에는 19.7%로 가장 낮아졌다가 이후 30%에서 40%로 서서히 증가하는 경향 이 나타났다. 계열별로는 신경정신계 후유증이 델타 우세기 고찰 에 27.4%, 2022년 1주차에 29.0%로 가장 발생률이 높았으나 이후 낮아져 15% 수준을 유지하였다. 반면 호흡기계 후유증 코로나19의 대유행으로 2022년 현재 우리나라 국민의 절 은 2022년 1월 4주부터 14.1%로 신경정신계 후유증보다 발 반 이상이 코로나19를 경험하였다는 점을 감안하면, ’코로나 생률이 높아진 이후 점차 발생률이 증가하여 5월 1주 37.3% 19 후유증’에 대한 진료지침 및 관리 정책의 개발은 공중보 에 이르렀다. 후각/미각장애의 경우 델타 시기에는 21.1%로 건의 관점에서 중요한 문제라고 할 수 있다. 문제는 이러한 높은 발생률을 보였으나 나머지 시기에는 그렇지 않았다. ‘코로나19 후유증’에 대한 명확한 개념과 정의가 부재하였다 Figure 2는 확진일에 따른 신경정신계 후유증 내의 개별 는 것이다. 코로나19 후유증(long COVID)이라는 명칭 외에 항목별 